In [1]:
import sys
from importlib import reload
import os

def reload_project_modules(project_dirs):
    """
    Reloads all modules that are located within the specified project directories.

    Parameters:
    - project_dirs: A list of root paths where project modules are located.

    Returns:
    - A list of module names that were successfully reloaded.
    """
    project_dirs = [os.path.abspath(path) for path in project_dirs]  # Convert to absolute paths
    reloaded_modules = []
    failed_modules = []

    for name, module in list(sys.modules.items()):
        try:
            if hasattr(module, '__file__'):
                module_path = os.path.abspath(module.__file__)
                module_dir = os.path.dirname(module_path)

                if any(module_dir.startswith(project_dir) for project_dir in project_dirs):
                    reload(module)
                    reloaded_modules.append(name)
            else:
                continue
        except Exception as e:
            failed_modules.append((name, str(e)))

    return reloaded_modules

In [2]:
import torch
from configs import DEPTH_MODEL_PATH, LR_MODEL_PATH, MR_MODEL_PATH
from models.yolo.utils import load_model

from utils.shared_resources import initialize_depth_model, initialize_vision_models

lr_model = load_model(LR_MODEL_PATH, device=torch.device('cuda:0'), half=True)
mr_model = load_model(MR_MODEL_PATH , device=torch.device('cuda:0'), half=True)
initialize_vision_models(lr_model, mr_model)

Loading checkpoint from ./checkpoints/best_ckpt_longrange.pt

Fusing model...
Loading checkpoint from ./checkpoints/best_ckpt_midrange.pt

Fusing model...


In [3]:
from models.depth_anything.utils import get_depth_model

depth_anything = get_depth_model('zoedepth', DEPTH_MODEL_PATH)
initialize_depth_model(depth_anything)

xFormers not available
xFormers not available
using MLP layer as FFN


Params passed to Resize transform:
	width:  640
	height:  640
	resize_target:  True
	keep_aspect_ratio:  False
	ensure_multiple_of:  14
	resize_method:  minimal
Using pretrained resource local::./checkpoints/depth_anything_metric_depth_outdoor.pt
Loaded successfully


In [14]:
import geopandas as gpd
from shapely import Point, LineString, Polygon

gdf = gpd.read_file('./assets/examples/shapefile/curitiba_updated.shp')
gdf

def set_z_to_zero(geom):
    """ Set the Z value of a geometry object to zero. """
    if geom.is_empty:
        return geom
    elif isinstance(geom, Point):
        return Point([geom.x, geom.y, 0])
    elif isinstance(geom, LineString) or isinstance(geom, Polygon):
        return type(geom)([(pt.x, pt.y, 0) for pt in geom.coords])
    return geom

# Apply the function to set Z to zero
gdf['geometry'] = gdf['geometry'].apply(set_z_to_zero)
gdf

,Name,descriptio,timestamp,begin,end,altitudeMo,tessellate,extrude,visibility,drawOrder,icon,geometry
0,1,None,None,None,None,None,-1,0,-1,None,None,POINT Z (-49.30775 -25.43757 0.00000)
1,2,None,None,None,None,None,-1,0,-1,None,None,POINT Z (-49.30793 -25.43765 0.00000)
2,3,None,None,None,None,None,-1,0,-1,None,None,POINT Z (-49.30766 -25.43785 0.00000)
3,Marcador de local sem título,None,None,None,None,None,-1,0,-1,None,None,POINT Z (-49.30778 -25.43796 0.00000)
4,Marcador de local sem título,None,None,None,None,None,-1,0,-1,None,None,POINT Z (-49.30772 -25.43818 0.00000)
...,...,...,...,...,...,...,...,...,...,...,...,...
80,Marcador de local sem título,None,None,None,None,None,-1,0,-1,None,None,POINT Z (-49.30364 -25.43706 0.00000)
81,Marcador de local sem título,None,None,None,None,None,-1,0,-1,None,None,POINT Z (-49.30634 -25.43792 0.00000)
82,Marcador de local sem título,None,None,None,None,None,-1,0,-1,None,None,POINT Z (-49.30840 -25.43771 0.00000)
83,Marcador de local sem título,None,None,None,None,None,-1,0,-1,None,None,POINT Z (-49.30812 -25.43720 0.00000)


In [15]:
import pandas as pd

selected = pd.read_csv('./assets/examples/sample_curitiba.csv')
selected['geometry'] = gpd.points_from_xy(x=selected['LONGITUDE'], y=selected['LATITUDE'])
selected = selected.iloc[:3]
selected

,LATITUDE,LONGITUDE,geometry
0,-25.437271,-49.303739,POINT (-49.30374 -25.43727)
1,-25.437249,-49.303583,POINT (-49.30358 -25.43725)
2,-25.437098,-49.303656,POINT (-49.30366 -25.43710)


In [6]:
import core.world
import pandas as pd

world = core.world.World()

selected = pd.read_csv('./assets/examples/sample_curitiba.csv')
selected['geometry'] = gpd.points_from_xy(x=selected['LONGITUDE'], y=selected['LATITUDE'])
selected = gpd.GeoDataFrame(selected, geometry='geometry')
selected = selected.iloc[:3]

for lat, lon in selected[['LATITUDE', 'LONGITUDE']].values:
    world.add_Location360(lat=lat, lon=lon, download=False, walk=False)    

world.locations

In [7]:
for location in world.locations:
    location.download_starters()
    location.generate_walks(world=world)


Generating Walks for Starter 0
Creating a walk for an object
Generating Walks for Starter 1
Creating a walk for an object
Creating a walk for an object
No movement planned, reusing image and setting image type to final_step
no more movement planned, stopping walk and fetching final images
Creating a walk for an object
Generating Walks for Starter 2
Creating a walk for an object
No movement planned, reusing image and setting image type to final_step
no more movement planned, stopping walk and fetching final images
Creating a walk for an object
Generating Walks for Starter 3
Creating a walk for an object
Creating a walk for an object
Generating Walks for Starter 0
Creating a walk for an object
After a step, no objects were detected. Stopping the walk.
Walk was not successful. Cutting it Short on step = 0
Generating Walks for Starter 1
Creating a walk for an object
Generating Walks for Starter 2
Generating Walks for Starter 3
Creating a walk for an object
During the walk, I suspected that

In [ ]:
world.locations[0].starters[0].walks[0].traverse(threshold=0.5)

In [10]:
world_config = {'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': ['objects'],
     'id': 'o3yn4nfz',
     'name': ['target'],
     'type': 'select',
     'value': True,
     'enlarged': False,
     'plotType': 'histogram',
     'animationWindow': 'free',
     'yAxis': None,
     'speed': 1},
    {'dataId': ['objects'],
     'id': 'ieifk7u9d',
     'name': ['image_type'],
     'type': 'multiSelect',
     'value': ['final_step'],
     'enlarged': False,
     'plotType': 'histogram',
     'animationWindow': 'free',
     'yAxis': None,
     'speed': 1}],
   'layers': [{'id': 'oyar70g',
     'type': 'geojson',
     'config': {'dataId': 'objects',
      'label': 'objects',
      'color': [41, 76, 181],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.5,
       'strokeOpacity': 0.8,
       'thickness': 2.5,
       'strokeColor': None,
       'colorRange': {'name': 'ColorBrewer Dark2-7',
        'type': 'qualitative',
        'category': 'ColorBrewer',
        'colors': ['#1b9e77',
         '#d95f02',
         '#7570b3',
         '#e7298a',
         '#66a61e',
         '#e6ab02',
         '#a6761d']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': False,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'obj_type', 'type': 'integer'},
      'colorScale': 'quantize',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}},
    {'id': 'ittjroo',
     'type': 'geojson',
     'config': {'dataId': 'Postes Vtal',
      'label': 'Postes Vtal',
      'color': [210, 0, 0],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.5,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': False,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'objects': [{'name': 'id',
        'format': None},
       {'name': 'distance', 'format': None},
       {'name': 'image_type', 'format': None},
       {'name': 'target', 'format': None}],
      'Postes Vtal': [{'name': 'Name', 'format': None},
       {'name': 'descriptio', 'format': None},
       {'name': 'timestamp', 'format': None},
       {'name': 'begin', 'format': None},
       {'name': 'end', 'format': None}],
      'Starters': [{'name': 'LATITUDE', 'format': None},
       {'name': 'LONGITUDE', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': -25.437284343802318,
   'longitude': -49.306400994975846,
   'pitch': 0,
   'zoom': 16.19470706392811,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

In [17]:
from keplergl import KeplerGl

def plot_objects(world, map_=None, config=None):
        if not map_:
            map_ = KeplerGl(height=600)

        tmp_df = world.gdf.copy()
        map_.add_data(data=tmp_df.drop(columns=['obj', 'image', 'starter']), name='objects')

        if config:
            map_.config = config

        return map_

world_map = plot_objects(world, config=world_config)
world_map.add_data(data=gdf, name='Postes')
world_map.add_data(data=gpd.GeoDataFrame(selected, geometry='geometry'), name='Starters')
world_map

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['objects'], 'id': 'o3yn4nfz'…